In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

ModuleNotFoundError: No module named 'keras'

In [8]:
classes = ['ソラスズメダイ', 'キンギョハナダイ', 'カゴカキダイ', 'ガラスハゼ', 'トラウツボ', 'ミツボシクロスズメダイ', 'マツカサウオ',
           'カミソリウオ', 'ミノカサゴ', 'イシガキフグ']
num_classes = len(classes)
image_size = 50

In [ ]:
X_train, X_test, Y_train, Y_test = np.load('./fish_aug5.npy', allow_pickle=True)
X_train = X_train.astype('float') / 256
X_test  = X_test.astype('float') / 256
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test  = np_utils.to_categorical(Y_test, num_classes)

In [ ]:
def model_train(X, Y, X_test, Y_test):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10)) # number of images
    model.add(Activation('softmax'))

    #opt = keras.optimizers.Adam()
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
                    optimizer=opt, metrics=['accuracy'])

    model.fit(X, Y, batch_size=32, epochs=200,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                       min_delta=0, patience=100,
                                                       verbose=1)],
              validation_data=(X_test, Y_test))

    # save model
    model.save('./fish_cnn_aug5.h5')

    return model

In [ ]:
def model_eval(model, X, Y):
    scores = model.evaluate(X, Y, verbose=1)
    print('Test loss: ', scores[0])
    print('Test Accuracy: ', scores[1])

In [ ]:
model = model_train(X_train, Y_train, X_test, Y_test)
model_eval(model, X_test, Y_test)

In [7]:
# 計算結果：８４%

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)       